In [ ]:
import boto3
import polars as pl
import botocore
import tarfile
import os
from io import BytesIO
from pypdf import PdfReader, PdfWriter
import gzip
from io import BytesIO
from fastwarc import ArchiveIterator
from fastwarc.stream_io import GZipStream
from resiliparse.extract.html2text import extract_plain_text
from resiliparse.parse.lang import detect_fast
from resiliparse.parse.html import HTMLTree
from fastwarc.warc import is_http
from surt import surt
import tldextract
import idna
import re
from urllib.parse import urljoin, urlparse
from resiliparse.parse.encoding import detect_encoding, bytes_to_str
import boto3
import polars as pl
from fastwarc.warc import ArchiveIterator, WarcRecordType
from fastwarc.stream_io import GZipStream
import gcsfs # Ensure you have this installed for GCS filesystem handling
import os
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())

client = boto3.client('s3', 
                      aws_access_key_id=os.getenv('ASCII_AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('ASCII_AWS_SECRET_ACCESS_KEY'))
#client = boto3.client('s3', 
#        region_name='us-east-1',  # Common Crawl is in us-east-1
#        config=Config(signature_version=UNSIGNED)
#    )


NameError: name 'GCP_PROJECT' is not defined

In [2]:
crawls = [
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251115220136-05256.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251115234907-05257.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116014712-05258.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116034034-05259.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116054722-05260.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116072531-05261.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116085944-05262.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116103419-05263.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116120640-05264.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116134805-05265.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116152516-05266.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116170848-05267.warc.gz',
'crawl-data/CC-NEWS/2025/11/CC-NEWS-20251116190217-05268.warc.gz']
key = 'crawl-data/CC-NEWS/index.html'
response = client.get_object(Bucket='commoncrawl', Key=key)

In [3]:
response['Body'].read()

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <title>Common Crawl News Dataset</title>\n  <meta charset="utf-8">\n  <link rel="stylesheet" href="https://data.commoncrawl.org/static/bucket.css">\n  <style>\n    th,td { padding: 0.2em 0.5em; }\n    th { background-color: #e0e0e0; }\n    tr { background-color: #f0f0f0; }\n    tr:nth-child(odd) { background-color: #fcfcfc; }\n    footer { width: 100%; margin-top: 1em; }\n  </style>\n</head>\n\n<body>\n\n<h1>Common Crawl News Dataset</h1>\n\n<p>The news dataset includes articles from news sites all over the world. WARC files are released on a daily basis. The news crawl was started in 2016, please see the <a href="https://commoncrawl.org/2016/10/news-dataset-available/">news dataset announcement</a> for further information.</p>\n\n<p>The <a href="https://github.com/commoncrawl/news-crawl/">source code of the news crawler</a> is available on our <a href="https://github.com/commoncrawl/">GitHub account</a>.\n\nPlease, <a href="https://github

In [4]:
key = 'crawl-data/CC-NEWS/2025/index.html'
response = client.get_object(Bucket='commoncrawl', Key=key)

In [5]:
key_gz = 'crawl-data/CC-NEWS/2025/11/warc.paths.gz'

response = client.get_object(Bucket='commoncrawl', Key=key_gz)
    
gzipped_body_bytes = response['Body'].read()
    
decompressed_bytes = gzip.decompress(gzipped_body_bytes)


In [6]:
current_data = pl.read_csv(BytesIO(decompressed_bytes), has_header=False)
print(current_data.shape)
current_data.head()

(570, 1)


column_1
str
"""crawl-data/CC-NEWS/2025/11/CC-…"
"""crawl-data/CC-NEWS/2025/11/CC-…"
"""crawl-data/CC-NEWS/2025/11/CC-…"
"""crawl-data/CC-NEWS/2025/11/CC-…"
"""crawl-data/CC-NEWS/2025/11/CC-…"


In [24]:
data = data.with_columns((pl.lit('crawl-data/CC-NEWS/')+pl.col('year')+pl.lit('/')+pl.col('month')+pl.lit('/')+pl.col('filename')).alias('column_1'))
data.join(current_data, on='column_1')


year,month,filename,column_1
str,str,str,str
"""2025""","""11""","""CC-NEWS-20251101004549-04954.w…","""crawl-data/CC-NEWS/2025/11/CC-…"
"""2025""","""11""","""CC-NEWS-20251101021758-04955.w…","""crawl-data/CC-NEWS/2025/11/CC-…"
"""2025""","""11""","""CC-NEWS-20251101033728-04956.w…","""crawl-data/CC-NEWS/2025/11/CC-…"
"""2025""","""11""","""CC-NEWS-20251101050844-04957.w…","""crawl-data/CC-NEWS/2025/11/CC-…"
"""2025""","""11""","""CC-NEWS-20251101062720-04958.w…","""crawl-data/CC-NEWS/2025/11/CC-…"
…,…,…,…
"""2025""","""11""","""CC-NEWS-20251104201114-05032.w…","""crawl-data/CC-NEWS/2025/11/CC-…"
"""2025""","""11""","""CC-NEWS-20251104212027-05033.w…","""crawl-data/CC-NEWS/2025/11/CC-…"
"""2025""","""11""","""CC-NEWS-20251104222953-05034.w…","""crawl-data/CC-NEWS/2025/11/CC-…"


In [27]:
from datetime import datetime
for path in current_data['column_1'].to_list():
    print(path.split('/')[-1].split("-"))
    format_date = datetime.strptime(path.split('/')[-1].split("-")[2], '%Y%m%d%H%M%S')
    print(format_date)
    break

['CC', 'NEWS', '20251101004549', '04954.warc.gz']
2025-11-01 00:45:49


In [28]:
ip_pattern = re.compile(r"^(?:www\.)?\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\Z")
host_part_pattern = re.compile(
    r"^[a-z0-9]([a-z0-9_-]{0,61}[a-z0-9])?\Z", re.IGNORECASE | re.ASCII
)


In [29]:
def get_surt_host(url):  # noqa: C901
    extracted = tldextract.extract(url, include_psl_private_domains=True)
    registered_domain = extracted.top_domain_under_public_suffix

    if registered_domain == "":
        registered_domain = f"{extracted.subdomain}.{extracted.domain}"
        if registered_domain == "":
            try:
                # Fallback to urlparse if tldextract fails
                host = urlparse(url).hostname
            except Exception as e:
                print(f"Failed to parse URL {url}: {e}")
                return None
            if not host:
                return None
        else:
            host = registered_domain
    else:
        host = registered_domain

    host = host.strip().lower()
    if len(host) < 1 or len(host) > 253:
        return None
    if ip_pattern.match(host):
        return None
    parts = host.split(".")
    if parts[-1] == "":
        # trailing dot is allowed, strip it
        parts = parts[0:-1]
    if len(parts) <= 1:
        # do not accept single-word hosts, must be at least `domain.tld'
        return None
    if len(parts) > 2 and parts[0] == "www":
        # strip leading 'www' to reduce number of "duplicate" hosts,
        # but leave at least 2 trailing parts (www.com is a valid domain)
        parts = parts[1:]
    for i, part in enumerate(parts):
        if len(part) > 63:
            return None
        if not host_part_pattern.match(part):
            try:
                idn = idna.encode(part).decode("ascii")
            except (
                idna.IDNAError,
                idna.core.InvalidCodepoint,
                UnicodeError,
                IndexError,
                Exception,
            ):
                print("Invalid host name: {}".format(url))
                return None

            # TODO: idna verifies the resulting string for length restrictions or invalid chars,
            #       maybe no further verification is required:
            if host_part_pattern.match(idn):
                parts[i] = idn
            else:
                print("Invalid host name: {}".format(url))
                return None
    parts.reverse()
    return ".".join(parts)

In [ ]:
collected_urls = (
    pl.scan_parquet('gs://gen-ai-tu/news/raw/', 
                    storage_options={
                        "project_id": os.getenv('GCP_PROJECT'),
                        "private_key_id": os.getenv('GCP_PRIVATE_KEY_ID'),
                        "private_key": os.getenv('GCP_PRIVATE_KEY'),
                        "client_email": os.getenv('GCP_EMAIL')})
    .select(['year', 'month', 'filename'])
    .unique()
    .head(100)
    .collect()
    .with_columns((pl.lit('crawl-data/CC-NEWS/')+pl.col('year')+pl.lit('/')+pl.col('month')+pl.lit('/')+pl.col('filename')).alias('column_1'))
    ['column_1'].to_list()
    )
output_path = 'gs://gen-ai-tu/news/raw/'
for key in current_data['column_1'].to_list():
    if key in collected_urls:
        print(f'{key}: was collected before')
        continue    
    print(f"Processing {key}...")
    try:
        # --- FIX 1: Initialize stream INSIDE the loop ---
        s3_response = client.get_object(Bucket='commoncrawl', Key=key)
        s3_stream = s3_response['Body']
        stream = GZipStream(s3_stream)
        
        tmp = []
        for record in ArchiveIterator(stream, record_types=WarcRecordType.response, func_filter=is_http):
            try:
                uri = record.headers.get('WARC-Target-URI')
                body_bytes = record.reader.read()
                html = bytes_to_str(body_bytes, detect_encoding(body_bytes))
                text = extract_plain_text(html)                
                http_date = record.http_date
                http_last_modified = record.http_last_modified
                if isinstance(http_date, datetime):
                    http_date = http_date.isoformat()
                if isinstance(http_last_modified, datetime):
                    http_last_modified = http_last_modified.isoformat()
                http_charset= record.http_charset 
                surt_uri = surt(uri)
                host = get_surt_host(uri)
                r = detect_fast(text, n_results=3)                
                if not r:
                    main_lang = "und"  # undefined
                    langs = []
                    confs = []
                else:
                    main_lang = r[0][0]
                    langs = [lang for (lang, conf) in r]
                    confs = [conf for (lang, conf) in r]
                tmp.append({
                    'uri': uri,
                    'tree': html,
                    'text': text,
                    'main_lang': main_lang,
                    'langs': langs, 
                    'confs': confs,
                    'http_date': http_date,
                    'http_last_modified': http_last_modified,
                    'http_charset': http_charset,
                    'surt_uri': surt_uri,
                    'host': host})
            except Exception as e:
                print(f"Error processing record: {e}")
        if not tmp:
            print(f"No valid records found in {key}, skipping write.")
            continue
        try:
            path_parts = key.split('/')
            filename = path_parts[-1]
            year_str = path_parts[-3]  # '2025'
            month_str = path_parts[-2] # '11'
            timestamp = filename.split('-')[2] # '20251116190217'
            day_str = timestamp[6:8]
            day_str

            df = pl.from_dicts(tmp).with_columns(
                pl.lit(filename).alias('filename'),
                pl.lit(year_str).alias('year'),
                pl.lit(month_str).alias('month'),
                pl.lit(day_str).alias('day')
            )
        
            df.write_parquet(
                output_path,
                partition_by=['year', 'month', 'day', 'main_lang', 'filename'],
                #use_pyarrow=True ,
                storage_options={
                    "project_id": os.getenv('GCP_PROJECT'),
                    "private_key_id": os.getenv('GCP_PRIVATE_KEY_ID'),
                    "private_key": os.getenv('GCP_PRIVATE_KEY'),
                    "client_email": os.getenv('GCP_EMAIL')},
                mkdir=True,
            )
            print(f"Done with {key}")
        except Exception as write_err:
            print(f"Failed to write parquet to GCS for {key}: {write_err}")
    except Exception as e:
        print(f"Error processing key {key}: {e}")

crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101004549-04954.warc.gz: was collected before
crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101021758-04955.warc.gz: was collected before
crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101033728-04956.warc.gz: was collected before
crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101050844-04957.warc.gz: was collected before
crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101062720-04958.warc.gz: was collected before
crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101074035-04959.warc.gz: was collected before
crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101085453-04960.warc.gz: was collected before
crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101100459-04961.warc.gz: was collected before
crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101110747-04962.warc.gz: was collected before
crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101120938-04963.warc.gz: was collected before
crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101132024-04964.warc.gz: was collected before
crawl-data/CC-NEWS/2025/11/CC-NEWS-20251101143201-0496